In [ ]:
!pip install torch==2.4.1
!pip install dgl==0.9.0
!pip install torch_geometric
!pip install tensorflow
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import os
from natsort import natsorted
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import FastText
import matplotlib.pyplot as plt
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense,Bidirectional
from tensorflow.keras.regularizers import l2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import numpy as np
from numpy.linalg import norm
from dgl.nn import GraphConv
from IPython.display import FileLink

import os
import zipfile
import numpy as np
from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
class MySentences:
    def __init__(self, zip_file_path, extract_to):
        self.zip_file_path = zip_file_path
        self.extract_to = extract_to
        self.root_directory = self.unzip_and_get_root_directory()

    def unzip_and_get_root_directory(self):
        with zipfile.ZipFile(self.zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(self.extract_to)
        return os.path.join(self.extract_to, 'Image_data')  # Adjust if folder name is different

    def __iter__(self):
        file_list = [file for file in os.listdir(self.root_directory) if not file.startswith('.DS_Store')]
        file_list = natsorted(file_list)

        for subdirectory in file_list:
            subdirectory_path = os.path.join(self.root_directory, subdirectory, 'txt')
            if os.path.exists(subdirectory_path):
                file_list_t = [file for file in os.listdir(subdirectory_path) if not file.startswith('.DS_Store')]
                file_list_t = natsorted(file_list_t)

                for filename in file_list_t:
                    file_path = os.path.join(subdirectory_path, filename)
                    with open(file_path, 'r') as file:
                        A = []
                        for line in file:
                            line = line.lower()
                            line = pun_remove(line)
                            line = word_tokenize(line)
                            line_Without_stopwords = [word for word in line if word not in stopwords_list]
                            if line_Without_stopwords:
                                A.append(line_Without_stopwords)
                        A = list(np.concatenate(A))
                        yield ' '.join(A)  # Combine tokens into a sentence


In [ ]:
# Initialize BERT model and tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Define function to get sentence embeddings from BERT
def get_bert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the embeddings for the [CLS] token (sentence-level representation)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Usage
zip_file_path = '/Image_data.zip'
extract_to = '/Extracted_image_data'

# Create an instance of MySentences with the zip file path
sentences = MySentences(zip_file_path, extract_to)

# Iterate over the sentences, extract BERT embeddings, and collect them in a DataFrame
embeddings = []
for sentence in sentences:
    embedding = get_bert_embedding(sentence)
    embeddings.append(embedding)

# Create a DataFrame with BERT embeddings
X_train_df = pd.DataFrame(embeddings)

# Save the DataFrame to a CSV file
X_train_df.to_csv('Text_features_bert.csv', index=False)

# Display download link for the CSV file
from IPython.display import FileLink
FileLink('Text_features_bert.csv')

/content/Text_features_bert.csv

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords_list = stopwords.words('english')
stopwords_list.append('33')
stopwords_list.append('52')
stopwords_list


remove=string.punctuation
new_symbol = '’'
new_symbol1 = '‘'
new_symbol2 = '–'
new_symbol3 = '—'

remove = remove + new_symbol
remove = remove + new_symbol1
remove = remove + new_symbol2
remove = remove + new_symbol3



def pun_remove(data):
    for char in remove:
        data=data.replace(char,' ')
    data=data.replace('“','')
    data=data.replace('”','')
    return data

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Preparing Node Data
Nodes_Data = pd.DataFrame({'Id': range(len(embeddings)), 'features': embeddings})
labels = pd.read_csv("/AnimalLabels.csv")
labels['majority_vote'] = labels.mode(axis=1, numeric_only=True).astype(int)
Nodes_Data['label'] = labels['majority_vote']

# Generating Edge Data for pairwise nodes with cosine similarity
Src_ID, Dst_ID, Src_feature, Dst_feature, edge_weight = [], [], [], [], []
for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        Src_ID.append(i)
        Dst_ID.append(j)
        src_feat, dst_feat = np.array(embeddings[i]), np.array(embeddings[j])
        similarity = np.dot(src_feat, dst_feat) / (norm(src_feat) * norm(dst_feat))
        Src_feature.append(src_feat)
        Dst_feature.append(dst_feat)
        edge_weight.append(max(similarity, 0))  # Ensure non-negative weights

Edge_Data = pd.DataFrame({
    'Src Id': Src_ID,
    'Dst Id': Dst_ID,
    'Src_feature': Src_feature,
    'Dst_feature': Dst_feature,
    'edge weights': edge_weight
})

# Save edge data to CSV
Edge_Data.to_csv('Edge_Data.csv', index=False)

# Provide download links
from IPython.display import FileLink
FileLink('Text_features_bert.csv'), FileLink('Edge_Data.csv')

(/content/Text_features_bert.csv, /content/Edge_Data.csv)

In [ ]:
print(Edge_Data['edge weights'].describe())
ninetyfive_percentile = Edge_Data['edge weights'].quantile(0.95)
print("95th percentile of edge weights column:", ninetyfive_percentile)

count    5565.000000
mean        0.897511
std         0.048203
min         0.580308
25%         0.877725
50%         0.909593
75%         0.929596
max         1.000000
Name: edge weights, dtype: float64
95th percentile of edge weights column: 0.9513047099113464


In [ ]:
percentiles=[round(0.877725,4),round(0.909593,4),round(0.929596,4),round(0.9513047099113464,4)]

In [ ]:
import os
os.environ["DGLBACKEND"] = "pytorch"

import dgl
import torch
import numpy as np
from dgl.data import DGLDataset

# Assuming `Nodes_Data` and `Edge_Data` DataFrames are already defined

class CustomGraphDataset(DGLDataset):
    def __init__(self, threshold):
        self.threshold = threshold
        super().__init__(name="custom_graph")

    def process(self):
        # Define the list to store indices of edges to be removed
        edge_remove = []

        # Extract edge weights and apply threshold filtering
        edge_weights = Edge_Data["edge weights"].to_numpy()
        for i in range(len(edge_weights)):
            if edge_weights[i] <= self.threshold:
                edge_remove.append(i)
            else:
                edge_weight.append(edge_weights[i])

        # Convert node features and labels
        features_array = np.array(Nodes_Data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array).float()
        node_labels = torch.tensor(Nodes_Data["label"].values, dtype=torch.long)

        # Convert edge features and source/destination IDs
        edge_features = torch.tensor(edge_weights, dtype=torch.float32)
        edges_src = torch.tensor(Edge_Data["Src Id"].values, dtype=torch.long)
        edges_dst = torch.tensor(Edge_Data["Dst Id"].values, dtype=torch.long)

        # Create the DGL graph
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=Nodes_Data.shape[0])
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # Remove edges below threshold
        self.graph = dgl.remove_edges(self.graph, torch.tensor(edge_remove, dtype=torch.long))

        # Create train, validation, and test masks
        n_nodes = Nodes_Data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True

        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, idx):
        return self.graph

    def __len__(self):
        return 1

# Set threshold value
threshold = 0.9275
dataset = CustomGraphDataset(threshold)
graph = dataset[0]

# Print the graph to confirm its structure
print(graph)


Graph(num_nodes=106, num_edges=1548,
      ndata_schemes={'feat': Scheme(shape=(768,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)})


In [ ]:
# Verify column names in Edge_Data
print("Edge_Data columns:", Edge_Data.columns)

# Adjust column names here if necessary
src_col = 'Src Id'  # Replace with the actual name for source column if different
dst_col = 'Dst Id'  # Replace with the actual name for destination column if different
edge_weight_col = 'edge weights'  # Replace with the actual name for edge weight column if different

# Updated Code with Column Names Adjusted
new_g = dgl.compact_graphs(graph)
train_mask = graph.ndata["train_mask"]

# Set train_mask to True for the first 106 nodes
for i in range(106):
    train_mask[i] = True

# Set specified indices to False in train_mask
indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
train_mask[indices_to_change] = False
graph.ndata["train_mask"] = train_mask

# Set test_mask as the inverse of train_mask
test_mask = ~train_mask
graph.ndata["test_mask"] = test_mask

# Filter Edge_Data for training edges (exclude edges connecting nodes in indices_to_change)
Edge_Data_train = Edge_Data[
    ~Edge_Data[src_col].isin(indices_to_change) &
    ~Edge_Data[dst_col].isin(indices_to_change) &
    (Edge_Data[edge_weight_col] > 0.8258)
]

# Filter Edge_Data for testing edges (include only edges connecting nodes in indices_to_change)
Edge_Data_test = Edge_Data[
    Edge_Data[src_col].isin(indices_to_change) &
    Edge_Data[dst_col].isin(indices_to_change) &
    (Edge_Data[edge_weight_col] > 0.8258)
]

# Create and save the dense adjacency matrix for the graph
adj_matrix = graph.adj(etype=None)
dense_matrix = adj_matrix.to_dense().numpy()

# Add node index as a new column to dense_matrix
new_column = np.arange(0, dense_matrix.shape[0]).reshape((106, 1))
dense_matrix = np.concatenate((new_column, dense_matrix), axis=1)

# Add row index as a new row to the top of dense_matrix
new_row = np.insert(np.arange(0, 106), 0, 0).reshape((1, 107))
result_matrix = np.vstack([new_row, dense_matrix])

# Save dense matrix to CSV
np.savetxt('dense_matrix_avg.csv', result_matrix, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_avg.csv')

# Generate subgraph for training set and save its adjacency matrix
sg_train = dgl.node_subgraph(graph, train_mask)
sg_adjacency_train = sg_train.adj()
dense_matrix_sg_train = sg_adjacency_train.to_dense().numpy()

np.savetxt('dense_matrix_sg_train.csv', dense_matrix_sg_train, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_sg_train.csv')

# Generate subgraph for test set and save its adjacency matrix
sg_test = dgl.node_subgraph(graph, test_mask)
sg_adjacency_test = sg_test.adj()
dense_matrix_sg_test = sg_adjacency_test.to_dense().numpy()

np.savetxt('dense_matrix_sg_test.csv', dense_matrix_sg_test, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_sg_test.csv')


Edge_Data columns: Index(['Src Id', 'Dst Id', 'Src_feature', 'Dst_feature', 'edge weights'], dtype='object')


/content/dense_matrix_sg_test.csv

In [ ]:
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
# class GCN(nn.Module):
#     def __init__(self, in_feats, h_feats, num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GraphConv(in_feats, h_feats,norm='both')
#         self.conv2 = GraphConv(h_feats, num_classes)

#     def forward(self, g, in_feat):
#         in_feat = in_feat.float()
#         h = self.conv1(graph, in_feat)
#         h = F.leaky_relu(h)
#         h = self.conv2(graph, h)
#         return h

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, norm='both')
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        in_feat = in_feat.float()
        h = self.conv1(g, in_feat)
        h = F.leaky_relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
los=[]
train_accuracy=[]
test_accuracy=[]
epoch=[]

sg_train = dgl.add_self_loop(sg_train)
sg_test = dgl.add_self_loop(sg_test)

def train(sg_train, sg_test, model):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1)
    best_train_acc = 0
    best_test_acc = 0

    features_train = sg_train.ndata["feat"]
    labels = graph.ndata["label"]
    train_mask = graph.ndata["train_mask"]
    test_mask = graph.ndata["test_mask"]

    for e in range(1000):
        # Forward pass
        logits = model(sg_train, features_train)
        pred = logits.argmax(1)

        labels = graph.ndata["label"].long()
        loss = F.cross_entropy(logits, labels[train_mask])

        # Test set predictions
        with torch.no_grad():
            model.eval()
            logits_test = model(sg_test, sg_test.ndata["feat"])
            pred_test = logits_test.argmax(1)

        # Calculate accuracies
        train_acc = (pred == labels[train_mask]).float().mean()
        test_acc = (pred_test == labels[test_mask]).float().mean()

        # Track best train and test accuracy
        best_train_acc = max(best_train_acc, train_acc)
        best_test_acc = max(best_test_acc, test_acc)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Log metrics every 5 epochs
        if e % 5 == 0:
            epoch.append(e)
            los.append(round(loss.item(), 3))
            train_accuracy.append(round(train_acc.item(), 3))
            test_accuracy.append(round(test_acc.item(), 3))
            print(
                f"In epoch {e}, loss: {loss:.3f}, train acc: {train_acc:.3f} (train {best_train_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )

# Initialize and train the model
model = GCN(graph.ndata["feat"].shape[1], 22, 2)
train(sg_train, sg_test, model)

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \


In epoch 0, loss: 0.666, train acc: 0.635 (train 0.635), test acc: 0.667 (best 0.667)
In epoch 5, loss: 61.697, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 10, loss: 307.550, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 15, loss: 218.458, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 20, loss: 11.955, train acc: 0.576 (train 0.659), test acc: 0.429 (best 0.667)
In epoch 25, loss: 17.949, train acc: 0.659 (train 0.671), test acc: 0.619 (best 0.667)
In epoch 30, loss: 6.212, train acc: 0.729 (train 0.729), test acc: 0.714 (best 0.714)
In epoch 35, loss: 3.757, train acc: 0.671 (train 0.729), test acc: 0.667 (best 0.714)
In epoch 40, loss: 27.561, train acc: 0.694 (train 0.729), test acc: 0.667 (best 0.714)
In epoch 45, loss: 19.097, train acc: 0.694 (train 0.729), test acc: 0.667 (best 0.714)
In epoch 50, loss: 9.991, train acc: 0.718 (train 0.729), test acc: 0.667 (best 0.714)
In epoch 55, loss: 4.636, train acc:

In [ ]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import torch
from dgl.data import DGLDataset


class KarateClubDataset(DGLDataset):
    def __init__(self,params):
        self.params=params
        #self.threshold=threshold
        super().__init__(name="karate_club")
        self.process()
    def process(self):
        edge_remove=[]
        C=edge_weights
        for i in range(0,len(C)):
            if C[i]<=self.params['percentile']:
                edge_remove.append(i)
        nodes_data = Nodes_Data
        edges_data = Edge_Data
        features_array = np.array(nodes_data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array)
        node_labels = torch.from_numpy(
                      nodes_data["label"].astype("category").cat.codes.to_numpy()
                       ).clone().detach()
        edge_features = torch.from_numpy(edges_data["edge weights"].to_numpy())
        edges_src = torch.from_numpy(edges_data["Src Ids"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Dst Ids"].to_numpy())
        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        self.graph=dgl.remove_edges(self.graph, torch.tensor(edge_remove))
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train : n_train + n_val] = True
        test_mask[n_train + n_val :] = True
        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

In [ ]:
import os
import dgl
import torch
import numpy as np
from dgl.data import DGLDataset

os.environ["DGLBACKEND"] = "pytorch"

class KarateClubDataset(DGLDataset):
    def __init__(self, params):
        self.params = params
        super().__init__(name="karate_club")
        self.process()

    def process(self):
        # Edge removal based on the percentile value
        edge_remove = []
        C = edge_weights  # Assuming `edge_weights` is predefined elsewhere
        for i in range(len(C)):
            if C[i] <= self.params['percentile']:
                edge_remove.append(i)

        nodes_data = Nodes_Data  # Assuming `Nodes_Data` is a predefined dataframe
        edges_data = Edge_Data  # Assuming `Edge_Data` is a predefined dataframe

        # Prepare node features, labels, and edge features
        features_array = np.array(nodes_data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array)
        node_labels = torch.from_numpy(
            nodes_data["label"].astype("category").cat.codes.to_numpy()
        ).clone().detach()

        edge_features = torch.from_numpy(edges_data["edge weights"].to_numpy())
        edges_src = torch.from_numpy(edges_data["Src Ids"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Dst Ids"].to_numpy())

        # Create the graph
        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # Remove edges based on edge removal condition
        self.graph = dgl.remove_edges(self.graph, torch.tensor(edge_remove))

        # Add self-loops to prevent 0-in-degree error
        self.graph = dgl.add_self_loop(self.graph)

        # Splitting into train, validation, and test masks
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)

        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)

        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True

        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1


In [ ]:
# seed = 42
# torch.manual_seed(seed)
# random.seed(seed)
# np.random.seed(seed)


# class GCN(nn.Module):
#     def __init__(self,in_feats,params,num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GraphConv(in_feats, params['output_features'])
#         self.conv2 = GraphConv(params['output_features'], num_classes)

#     def forward(self, g,in_feat,params):
#         in_feat = in_feat.float()
#         h = self.conv1(g, in_feat)
#         h = getattr(F, params['activation'])(h)
#         h = self.conv2(g, h)
#         return h

class GCN(nn.Module):
    def __init__(self, in_feats, params, num_classes):
        super(GCN, self).__init__()
        # Initialize the layers based on provided params
        self.conv1 = GraphConv(in_feats, params['output_features'])
        self.conv2 = GraphConv(params['output_features'], num_classes)

    def forward(self, g, in_feat, params):
        # Ensure features are in float format
        in_feat = in_feat.float()

        # Perform first graph convolution
        h = self.conv1(g, in_feat)

        # Apply activation function based on `params`
        h = getattr(F, params['activation'])(h)  # Dynamic activation based on params

        # Perform second graph convolution
        h = self.conv2(g, h)
        return h

In [ ]:
# los=[]
# train_accuracy=[]
# test_accuracy=[]
# TT=[]
# CT=[]
# epoch=[]
# seed = 42
# torch.manual_seed(seed)
# random.seed(seed)
# np.random.seed(seed)
# import torch.optim as optim
# import torch.nn.functional as F
# def train(g,sg_train,sg_test, model,params):
#     model.train()
#     optimizer = getattr(optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
#     best_train_acc = 0
#     best_test_acc = 0
#     corresponding_test=0

#     features_train = sg_train.ndata["feat"]
#     labels = g.ndata["label"]
#     train_mask = g.ndata["train_mask"]
#     test_mask = g.ndata["test_mask"]


#     for e in range(100):
#         # Forward
#         logits = model(sg_train,features_train,params)


#         # Compute prediction
#         pred = logits.argmax(1)


#         labels = g.ndata["label"].long()
#         if params['loss'] == 'binary_cross_entropy':
#             loss = getattr(F, params['loss'])(torch.sigmoid(logits), labels[train_mask])
#         else:
#             loss = getattr(F, params['loss'])(logits, labels[train_mask])

#         with torch.no_grad():
#             model.eval()
#             logits_test = model(sg_test,sg_test.ndata["feat"],params)


#         pred_test = logits_test.argmax(1)

#         train_acc = (pred == labels[train_mask]).float().mean()
#         test_acc = (pred_test == labels[test_mask]).float().mean()

#         if best_train_acc < train_acc:
#             best_train_acc = train_acc
#             corresponding_test=test_acc
#         if best_test_acc < test_acc:
#             best_test_acc = test_acc


#         # Backward
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()


#         if e % 5 == 0:
#             epoch.append(e)
#             los.append(round(loss.item(),3))
#             train_accuracy.append(round(train_acc.item(),3))

#     TT.append({'train_accuracy':best_train_acc})
#     CT.append({'test_accuracy': corresponding_test})

#     print(f'Train Accuracy: {best_train_acc}, Test Accuracy: {corresponding_test}')
#     return best_train_acc

import torch.optim as optim
import torch.nn.functional as F

# Lists to store metrics for later use
los = []
train_accuracy = []
test_accuracy = []
TT = []
CT = []
epoch = []

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def train(g, sg_train, sg_test, model, params):
    model.train()

    # Optimizer setup
    optimizer = getattr(optim, params['optimizer'])(model.parameters(), lr=params['learning_rate'])

    # Initialize tracking variables for best accuracy
    best_train_acc = 0
    best_test_acc = 0
    corresponding_test = 0

    # Retrieve the relevant graph and node data
    features_train = sg_train.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    test_mask = g.ndata["test_mask"]

    # Training loop
    for e in range(100):
        # Forward pass
        logits = model(sg_train, features_train, params)

        # Compute prediction
        pred = logits.argmax(1)

        # Loss calculation
        if params['loss'] == 'binary_cross_entropy':
            loss = getattr(F, params['loss'])(torch.sigmoid(logits), labels[train_mask])
        else:
            loss = getattr(F, params['loss'])(logits, labels[train_mask])

        # Evaluate performance on the test graph
        with torch.no_grad():
            model.eval()
            logits_test = model(sg_test, sg_test.ndata["feat"], params)

        pred_test = logits_test.argmax(1)

        # Calculate training and testing accuracies
        train_acc = (pred == labels[train_mask]).float().mean()
        test_acc = (pred_test == labels[test_mask]).float().mean()

        # Track the best accuracies
        if best_train_acc < train_acc:
            best_train_acc = train_acc
            corresponding_test = test_acc
        if best_test_acc < test_acc:
            best_test_acc = test_acc

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging every 5 epochs
        if e % 5 == 0:
            epoch.append(e)
            los.append(round(loss.item(), 3))
            train_accuracy.append(round(train_acc.item(), 3))

    # Store results for later
    TT.append({'train_accuracy': best_train_acc})
    CT.append({'test_accuracy': corresponding_test})

    # Print final accuracy results
    print(f'Train Accuracy: {best_train_acc}, Test Accuracy: {corresponding_test}')

    return best_train_acc


In [ ]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
# def objective(trial):
#     percentile_values=[percentiles[0],percentiles[1],percentiles[2],percentiles[3]]
#     learning_rate=[0.01,0.1,1,10,100,200]
#     output_features=[10,12,14,16,18,20,22]
#     params={'output_features':trial.suggest_categorical('output_features',output_features),
#            'optimizer':trial.suggest_categorical('optimizer',["Adam", "RMSprop", "SGD"]),
#            'percentile':trial.suggest_categorical('percentile',percentile_values),
#            'learning_rate':trial.suggest_categorical('learning_rate',learning_rate),
#            'loss':trial.suggest_categorical('loss',['cross_entropy']),
#            'activation':trial.suggest_categorical('activation',['relu','selu','elu','leaky_relu','tanh'])}
#     dataset = KarateClubDataset(params)
#     g = dataset[0]
#     train_mask = g.ndata["train_mask"]
#     for i in range(106):
#         train_mask[i] = True

#     indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
#     train_mask[indices_to_change] = False
#     g.ndata["train_mask"]=train_mask
#     test_mask = ~train_mask
#     g.ndata["test_mask"]=test_mask
#     sg_train=dgl.node_subgraph(g, train_mask)
#     sg_test = dgl.node_subgraph(g, test_mask)
#     model=GCN(g.ndata["feat"].shape[1],params, 2)
#     accuracy=train(g,sg_train,sg_test,model,params)
#     return accuracy

import torch
import optuna
import dgl
from dgl.data import KarateClubDataset

def objective(trial):
    # Define the possible values for each hyperparameter
    percentile_values = [percentiles[0], percentiles[1], percentiles[2], percentiles[3]]
    learning_rate = [0.0001,0.001,0.01,0.1]
    output_features = [10, 12, 14, 16, 18, 20, 22]

    # # Define the params dictionary with values for hyperparameter optimization
    # params = {
    #     'output_features': trial.suggest_categorical('output_features', output_features),
    #     'optimizer': trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"]),
    #     'percentile': trial.suggest_categorical('percentile', percentile_values),
    #     'learning_rate': trial.suggest_categorical('learning_rate', learning_rate),
    #     'loss': trial.suggest_categorical('loss', ['cross_entropy']),
    #     'activation': trial.suggest_categorical('activation', ['relu', 'selu', 'elu', 'leaky_relu', 'tanh'])
    # }

    params = {
    'output_features': trial.suggest_categorical('output_features', output_features),
    'optimizer': trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"]),
    'percentile': trial.suggest_categorical('percentile', percentile_values),
    'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
    'loss': trial.suggest_categorical('loss', ['cross_entropy']),
    'activation': trial.suggest_categorical('activation', ['relu', 'selu', 'elu']),
    'dropout': trial.suggest_float('dropout', 0.2, 0.5),  # Adding dropout hyperparameter
}

    # Load dataset and setup graph
    dataset = KarateClubDataset()
    g = dataset[0]

    # Initialize node features as random values (if not already present in the dataset)
    if "feat" not in g.ndata:
        g.ndata["feat"] = torch.randn(g.number_of_nodes(), 10)  # Here, we use 10 as the feature dimension, but adjust as necessary

    # Add self-loop to avoid zero-in-degree nodes
    g = dgl.add_self_loop(g)

    # Initialize train_mask and test_mask
    g.ndata["train_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)
    g.ndata["test_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)

    # Modify train_mask for training set (set to True for the available nodes, up to g.number_of_nodes())
    train_mask = g.ndata["train_mask"]
    for i in range(g.number_of_nodes()):  # Use number of nodes instead of fixed 106
        train_mask[i] = True

    # Manually set indices to False for testing set (ensure indices are within bounds)
    indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
    # Filter out indices that exceed the available number of nodes
    indices_to_change = [i for i in indices_to_change if i < g.number_of_nodes()]
    train_mask[indices_to_change] = False
    g.ndata["train_mask"] = train_mask
    test_mask = ~train_mask
    g.ndata["test_mask"] = test_mask

    # Create subgraphs for training and testing
    sg_train = dgl.node_subgraph(g, train_mask)
    sg_test = dgl.node_subgraph(g, test_mask)

    # Initialize GCN model with parameters from the trial
    model = GCN(g.ndata["feat"].shape[1], params, 2)

    # Train the model and get the best accuracy
    accuracy = train(g, sg_train, sg_test, model, params)

    return accuracy


In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=2520)
import time

# Initialize the study
study = optuna.create_study(direction="maximize")

# Measure the total time for all trials
start_time = time.time()  # Start time before optimization

# Perform the optimization
study.optimize(objective, n_trials=2520)

end_time = time.time()  # End time after optimization

# Calculate total compute time
total_compute_time = end_time - start_time

print(f"Total computation time for all 2520 trials: {total_compute_time:.4f} seconds")

[I 2024-11-20 21:02:55,680] A new study created in memory with name: no-name-bba6e855-fac4-46de-a18c-df6e90eae6d5
[I 2024-11-20 21:02:57,255] Trial 0 finished with value: 0.9615384340286255 and parameters: {'output_features': 16, 'optimizer': 'RMSprop', 'percentile': 0.9096, 'learning_rate': 0.0006225643370203724, 'loss': 'cross_entropy', 'activation': 'selu', 'dropout': 0.4581528693487451}. Best is trial 0 with value: 0.9615384340286255.


Train Accuracy: 0.9615384340286255, Test Accuracy: 0.5


[I 2024-11-20 21:02:58,887] Trial 1 finished with value: 0.8461538553237915 and parameters: {'output_features': 14, 'optimizer': 'Adam', 'percentile': 0.9296, 'learning_rate': 1.5117904247595468e-05, 'loss': 'cross_entropy', 'activation': 'selu', 'dropout': 0.27259191129646476}. Best is trial 0 with value: 0.9615384340286255.


Train Accuracy: 0.8461538553237915, Test Accuracy: 0.625


[I 2024-11-20 21:03:00,035] Trial 2 finished with value: 0.9615384340286255 and parameters: {'output_features': 14, 'optimizer': 'RMSprop', 'percentile': 0.9513, 'learning_rate': 0.0009312905775206136, 'loss': 'cross_entropy', 'activation': 'elu', 'dropout': 0.254893069976857}. Best is trial 0 with value: 0.9615384340286255.


...

Train Accuracy: 0.9615384340286255, Test Accuracy: 0.375


[I 2024-11-20 21:38:46,229] Trial 2518 finished with value: 1.0 and parameters: {'output_features': 14, 'optimizer': 'Adam', 'percentile': 0.9513, 'learning_rate': 0.017308264261269204, 'loss': 'cross_entropy', 'activation': 'elu', 'dropout': 0.2286564292031086}. Best is trial 7 with value: 1.0.


Train Accuracy: 1.0, Test Accuracy: 0.625


[I 2024-11-20 21:38:47,063] Trial 2519 finished with value: 0.5384615659713745 and parameters: {'output_features': 20, 'optimizer': 'Adam', 'percentile': 0.8777, 'learning_rate': 1.5950948911837383e-05, 'loss': 'cross_entropy', 'activation': 'selu', 'dropout': 0.24359865628784988}. Best is trial 7 with value: 1.0.


Train Accuracy: 0.5384615659713745, Test Accuracy: 0.5
Total computation time for all 2520 trials: 2151.3820 seconds


In [27]:
import heapq

# Assuming CT is a list of dictionaries and each dictionary has a 'test_accuracy' key
top_ten_max_results = heapq.nlargest(10, enumerate(CT), key=lambda x: x[1]['test_accuracy'])

# Print the top ten results along with their indices
for index, result in top_ten_max_results:
    print(f'Index: {index}, Test Accuracy: {result["test_accuracy"]}')

Index: 596, Test Accuracy: 1.0
Index: 91, Test Accuracy: 0.875
Index: 308, Test Accuracy: 0.875
Index: 367, Test Accuracy: 0.875
Index: 441, Test Accuracy: 0.875
Index: 462, Test Accuracy: 0.875
Index: 485, Test Accuracy: 0.875
Index: 704, Test Accuracy: 0.875
Index: 745, Test Accuracy: 0.875
Index: 756, Test Accuracy: 0.875


In [28]:
import heapq

# Assuming CT is a list of dictionaries and each dictionary has a 'test_accuracy' key
top_ten_max_results = heapq.nlargest(20, enumerate(TT), key=lambda x: x[1]['train_accuracy'])

# Print the top ten results along with their indices
for index, result in top_ten_max_results:
    print(f'Index: {index}, Test Accuracy: {result["train_accuracy"]}')

Index: 7, Test Accuracy: 1.0
Index: 10, Test Accuracy: 1.0
Index: 11, Test Accuracy: 1.0
Index: 12, Test Accuracy: 1.0
Index: 13, Test Accuracy: 1.0
Index: 14, Test Accuracy: 1.0
Index: 15, Test Accuracy: 1.0
Index: 17, Test Accuracy: 1.0
Index: 18, Test Accuracy: 1.0
Index: 21, Test Accuracy: 1.0
Index: 22, Test Accuracy: 1.0
Index: 23, Test Accuracy: 1.0
Index: 24, Test Accuracy: 1.0
Index: 29, Test Accuracy: 1.0
Index: 31, Test Accuracy: 1.0
Index: 32, Test Accuracy: 1.0
Index: 33, Test Accuracy: 1.0
Index: 34, Test Accuracy: 1.0
Index: 36, Test Accuracy: 1.0
Index: 37, Test Accuracy: 1.0


In [29]:
train=[]
test=[]
for i in range(0,2520):
    train.append(round(TT[i]['train_accuracy'].item(),4))
    test.append(round(CT[i]['test_accuracy'].item(),4))

Final_results=pd.DataFrame()

Final_results['train']=train
Final_results['test']=test

In [30]:
pd.set_option('display.max_rows', None)
Final_results

,train,test
0,0.9615,0.500
1,0.8462,0.625
2,0.9615,0.500
3,0.4615,0.375
4,0.4231,0.500
5,0.7308,0.375
6,0.5769,0.375
7,1.0000,0.500
8,0.3077,0.375
9,0.5000,0.625


In [31]:
X_train_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.152275,-0.016306,-0.029671,-0.309308,-0.157704,0.084691,0.545346,0.162155,-0.303332,-0.263704,...,0.105067,-0.257130,0.102520,0.022993,0.141487,0.018533,-0.219166,-0.214960,0.406201,-0.058291
1,-0.203677,-0.038802,-0.110415,-0.200640,0.101055,-0.120789,0.620446,0.116061,-0.408456,-0.188784,...,0.056379,-0.172524,0.105956,-0.011191,-0.076223,-0.149535,-0.281255,-0.294813,0.228613,-0.326719
2,-0.117061,0.141994,-0.025613,-0.265151,-0.016182,-0.048967,0.654484,0.263107,-0.371275,-0.134465,...,0.144098,-0.147902,0.032702,-0.092698,0.060497,0.096999,-0.218485,-0.204414,0.295918,-0.243709
3,0.045934,0.097099,0.022639,-0.131106,-0.036220,-0.391055,0.448852,0.340934,-0.467579,-0.192682,...,0.087434,-0.275805,0.072272,0.021598,0.318817,0.070363,-0.184780,-0.309102,0.262471,-0.101312
4,-0.252365,-0.129211,-0.126388,-0.072765,-0.085733,0.005264,0.563219,0.041376,-0.216820,-0.164519,...,0.085790,-0.126716,0.206637,-0.117690,0.010854,0.123012,-0.129832,-0.316416,0.274867,-0.306691
5,-0.269758,0.049815,-0.224666,-0.151418,-0.063038,-0.072747,0.712746,0.082776,-0.246820,-0.171637,...,0.307743,-0.167472,0.223036,-0.023777,-0.081383,0.269083,-0.126693,-0.173768,0.361744,0.136812
6,-0.143368,-0.048379,-0.262562,-0.248739,-0.123375,-0.038999,0.698019,0.052606,-0.234069,-0.139208,...,0.150556,-0.093120,0.139624,-0.091530,-0.126044,0.225691,-0.300129,-0.235319,0.308440,-0.192666
7,-0.019782,-0.029809,0.015159,-0.394077,-0.109174,0.033728,0.633219,0.178142,-0.195174,-0.234271,...,0.069415,-0.229153,0.139866,-0.152288,0.037289,-0.103430,-0.180977,-0.291372,0.303987,-0.136188
8,-0.073113,-0.013757,-0.149179,-0.224732,-0.179300,-0.134749,0.563628,-0.023371,-0.028352,-0.029855,...,0.192879,0.009457,0.434482,-0.088411,-0.073078,-0.070340,-0.242699,-0.397278,0.347571,0.192785
9,-0.292569,-0.269340,0.144939,-0.085762,-0.006944,-0.122196,0.356322,0.236450,-0.145089,-0.072183,...,0.229317,-0.174704,0.412304,-0.175340,0.105623,-0.187353,-0.131315,-0.224592,0.218511,0.220681


In [32]:
labels = pd.read_csv("/AnimalLabels.csv")
labels['majority_vote'] = labels.mode(axis=1, numeric_only=True).astype(int)
type(labels['majority_vote'])

pandas.core.series.Series

In [33]:
X_train_df['label']=labels['majority_vote']
y=X_train_df['label']
X=X_train_df

In [34]:
X.drop(columns=['label'],axis=1,inplace=True)
X

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.152275,-0.016306,-0.029671,-0.309308,-0.157704,0.084691,0.545346,0.162155,-0.303332,-0.263704,...,0.105067,-0.257130,0.102520,0.022993,0.141487,0.018533,-0.219166,-0.214960,0.406201,-0.058291
1,-0.203677,-0.038802,-0.110415,-0.200640,0.101055,-0.120789,0.620446,0.116061,-0.408456,-0.188784,...,0.056379,-0.172524,0.105956,-0.011191,-0.076223,-0.149535,-0.281255,-0.294813,0.228613,-0.326719
2,-0.117061,0.141994,-0.025613,-0.265151,-0.016182,-0.048967,0.654484,0.263107,-0.371275,-0.134465,...,0.144098,-0.147902,0.032702,-0.092698,0.060497,0.096999,-0.218485,-0.204414,0.295918,-0.243709
3,0.045934,0.097099,0.022639,-0.131106,-0.036220,-0.391055,0.448852,0.340934,-0.467579,-0.192682,...,0.087434,-0.275805,0.072272,0.021598,0.318817,0.070363,-0.184780,-0.309102,0.262471,-0.101312
4,-0.252365,-0.129211,-0.126388,-0.072765,-0.085733,0.005264,0.563219,0.041376,-0.216820,-0.164519,...,0.085790,-0.126716,0.206637,-0.117690,0.010854,0.123012,-0.129832,-0.316416,0.274867,-0.306691
5,-0.269758,0.049815,-0.224666,-0.151418,-0.063038,-0.072747,0.712746,0.082776,-0.246820,-0.171637,...,0.307743,-0.167472,0.223036,-0.023777,-0.081383,0.269083,-0.126693,-0.173768,0.361744,0.136812
6,-0.143368,-0.048379,-0.262562,-0.248739,-0.123375,-0.038999,0.698019,0.052606,-0.234069,-0.139208,...,0.150556,-0.093120,0.139624,-0.091530,-0.126044,0.225691,-0.300129,-0.235319,0.308440,-0.192666
7,-0.019782,-0.029809,0.015159,-0.394077,-0.109174,0.033728,0.633219,0.178142,-0.195174,-0.234271,...,0.069415,-0.229153,0.139866,-0.152288,0.037289,-0.103430,-0.180977,-0.291372,0.303987,-0.136188
8,-0.073113,-0.013757,-0.149179,-0.224732,-0.179300,-0.134749,0.563628,-0.023371,-0.028352,-0.029855,...,0.192879,0.009457,0.434482,-0.088411,-0.073078,-0.070340,-0.242699,-0.397278,0.347571,0.192785
9,-0.292569,-0.269340,0.144939,-0.085762,-0.006944,-0.122196,0.356322,0.236450,-0.145089,-0.072183,...,0.229317,-0.174704,0.412304,-0.175340,0.105623,-0.187353,-0.131315,-0.224592,0.218511,0.220681


In [35]:
indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]

# Create a boolean mask to identify indices for the test set
test_mask = np.zeros(len(X), dtype=bool)
test_mask[indices_to_change] = True

# Split the dataset based on the boolean mask
X_train = X[~test_mask]
X_test = X[test_mask]
y_train = y[~test_mask]
y_test = y[test_mask]

In [36]:
param_grid = {'penalty':['l1','l2',None],'C': [0.1,2,2.5,3],
                   'solver':['liblinear','saga'],'max_iter':[500]}
LR=LogisticRegression(random_state=0)
grid_search_LR = GridSearchCV(estimator=LR,
                           param_grid=param_grid,
                           cv=3)

grid_search_LR.fit(X_train, y_train)
print("Best parameters found: ", grid_search_LR.best_params_)
print("Best Score: {}".format(grid_search_LR.best_score_))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which mean

Best parameters found:  {'C': 0.1, 'max_iter': 500, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score: 0.6469622331691297


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
12 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  F

In [37]:
param_grid = {'penalty':['elasticnet',None],'C': [0.1,2,2.5,3],
                   'solver':['saga'],'max_iter':[500],'l1_ratio':[0.5,0.2,0.7]}
LR=LogisticRegression(random_state=0)
grid_search_LR = GridSearchCV(estimator=LR,
                           param_grid=param_grid,
                           cv=2)
grid_search_LR.fit(X_train, y_train)
print("Best parameters found: ", grid_search_LR.best_params_)
print("Best Score: {}".format(grid_search_LR.best_score_))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The

Best parameters found:  {'C': 0.1, 'l1_ratio': 0.5, 'max_iter': 500, 'penalty': None, 'solver': 'saga'}
Best Score: 0.6943521594684385


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [38]:
param_grid = {'penalty':['l2',None],'C': [0.1,2,2.5,3],
                   'solver':['lbfgs','newton-cg','sag'],'max_iter':[500]}
LR=LogisticRegression(random_state=0)
grid_search_LR = GridSearchCV(estimator=LR,
                           param_grid=param_grid,
                           cv=3
                          )

grid_search_LR.fit(X_train, y_train)
print("Best parameters found: ", grid_search_LR.best_params_)
print("Best Score: {}".format(grid_search_LR.best_score_))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C

Best parameters found:  {'C': 0.1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Score: 0.6231527093596059


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [39]:
LR=LogisticRegression(C=2.5,penalty='elasticnet',solver='saga',max_iter=1000,l1_ratio= 0.2)
LR.fit(X_train,y_train)

LogisticRegression(C=2.5, l1_ratio=0.2, max_iter=1000, penalty='elasticnet',
                   solver='saga')

In [40]:
LR_pred = LR.predict(X_train)
acc_train=accuracy_score(y_train,LR_pred)
print('train_accuracy',acc_train)
LR_pred = LR.predict(X_test)
acc=accuracy_score(y_test,LR_pred)
print('test_accuracy',acc)

train_accuracy 0.9529411764705882
test_accuracy 0.7619047619047619


In [41]:
X_train_values = X_train.values
X_test_values = X_test.values
#X_valid_values = X_valid.values

# Reshape input data to 3D [samples, timesteps, features] so it'll fit the LSTM layer
# Use a timestep of 1.
X_train_reshaped = np.reshape(X_train_values, (X_train_values.shape[0], 1, X_train_values.shape[1]))
X_test_reshaped = np.reshape(X_test_values, (X_test_values.shape[0], 1, X_test_values.shape[1]))
#X_valid_reshaped = np.reshape(X_valid_values, (X_valid_values.shape[0], 1, X_valid_values.shape[1]))

# LSTM model
lstm = Sequential()
lstm.add(LSTM(100,return_sequences=True,activation='selu',input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
lstm.add(Dropout(0.7))

lstm.add(LSTM(50,activation='selu',return_sequences=False,kernel_regularizer=l2(0.01)))
lstm.add(Dropout(0.7))
lstm.add(Dense(1,activation='sigmoid'))  # Prediction of the next closing value


# Compile the model
lstm.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
lstm.fit(X_train_reshaped, y_train, epochs=1000, batch_size=32, verbose=1)

# Predicting and inverse transforming the predictions
y_pred_train_lstm = lstm.predict(X_train_reshaped)
y_pred_test_lstm = lstm.predict(X_test_reshaped)

#y_pred = scaler.inverse_transform(y_pred)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 2.0401
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.9694
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.8229 
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.7910
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.7372
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.6874
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.6285
Epoch 8/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.5923 
Epoch 9/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.5431
Epoch 10/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.4449
Epoch 11/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.4892
Epoch 12/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.4100 
Epoch 13/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.3903 
Epoch 14/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.3267 
Epoch 15/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.2721

In [42]:
y_pred_train_lstm = np.squeeze(y_pred_train_lstm)
y_pred_test_lstm = np.squeeze(y_pred_test_lstm)
y_pred_train_lstm = (y_pred_train_lstm > 0.5).astype(int)
acc_train=accuracy_score(y_train,y_pred_train_lstm)
print('train accuracy',acc_train)
y_pred_test_lstm = (y_pred_test_lstm > 0.5).astype(int)
acc_test=accuracy_score(y_test,y_pred_test_lstm)
print('test accuracy',acc_test)

train accuracy 1.0
test accuracy 0.8095238095238095


In [43]:
bilstm = Sequential()
bilstm.add(Bidirectional(LSTM(100, return_sequences=True,kernel_regularizer=l2(0.01)), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
#bilstm.add(Dropout(0.7))
bilstm.add(Bidirectional(LSTM(50, return_sequences=False,kernel_regularizer=l2(0.01))))
#bilstm.add(Dropout(0.7))
bilstm.add(Dense(1, activation='sigmoid',kernel_regularizer=l2(0.01)))  # Binary classification output

# Compile the model
bilstm.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
bilstm.fit(X_train_reshaped, y_train, epochs=500, batch_size=32, verbose=1)

# Predicting
y_pred_train_bilstm = bilstm.predict(X_train_reshaped)
y_pred_test_bilstm = bilstm.predict(X_test_reshaped)

Epoch 1/500


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 14.9216
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 13.3939
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 11.9891
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 10.7225
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 9.5500
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 8.5083
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 7.5821
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 6.7315
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 6.0101
Epoch 10/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 5.3233
Epoch 11/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 4.7532
Epoch 12/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 4.2156
Epoch 13/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 3.7732
Epoch 14/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 3.4022
Epoch 15/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 3.0389
Epoch 16/500
3/3 ━━━━━━━━━━

In [44]:
y_pred_train_bilstm = np.squeeze(y_pred_train_bilstm)
y_pred_test_bilstm = np.squeeze(y_pred_test_bilstm)

In [45]:
y_pred_train_bilstm = (y_pred_train_bilstm > 0.5).astype(int)
acc_train=accuracy_score(y_train,y_pred_train_bilstm)
print('train accuracy',acc_train)
y_pred_test_bilstm = (y_pred_test_bilstm > 0.5).astype(int)
acc_test=accuracy_score(y_test,y_pred_test_bilstm)
print('test accuracy',acc_test)

train accuracy 0.6470588235294118
test accuracy 0.6666666666666666
